In [1]:
import pandas as pd
import seaborn as sns

# Cargar datos (ejemplo)
df = sns.load_dataset('titanic')
print("--- Antes de la limpieza ---")
print(df.isnull().sum()) 

# 1. DIAGNÓSTICO
print("--- Datos faltantes por columna ---")
print(df.isnull().sum())

# 2. ELIMINAR (Cirugía)
# La columna 'deck' (cubierta) tiene demasiados huecos. La matamos.
# axis=1 (columna), thresh=500 (borrar si tiene menos de 500 datos válidos)
df_limpio = df.dropna(axis=1, thresh=500)

# 3. RELLENAR NÚMEROS (Media)
# A 'age' le faltan datos, pero es importante. Rellenamos con el promedio.
promedio_edad = df['age'].mean()
df['age'].fillna(promedio_edad, inplace=True)

# 4. RELLENAR CATEGORÍAS (Moda)
# A 'embark_town' le faltan poquitos. Asumimos la ciudad más común.
ciudad_comun = df['embark_town'].value_counts().idxmax()
df['embark_town'].fillna(ciudad_comun, inplace=True)

print("\n--- Después de la limpieza ---")
print(df.isnull().sum()) # Debería salir casi todo en 0

--- Antes de la limpieza ---
survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64
--- Datos faltantes por columna ---
survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

--- Después de la limpieza ---
survived         0
pclass           0
sex              0
age              0
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      0
alive            0
alone            0
dtype: int64


C:\Users\alere\AppData\Local\Temp\ipykernel_21092\94053324.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['age'].fillna(promedio_edad, inplace=True)
C:\Users\alere\AppData\Local\Temp\ipykernel_21092\94053324.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example,

In [3]:
import pandas as pd
import seaborn as sns
import numpy as np

# --- 1. CARGA ---
df = sns.load_dataset('titanic')
print("--- 1. Datos Faltantes Iniciales ---")
print(df.isnull().sum())

# --- 2. LIMPIEZA (Sin Warnings) ---

# A) Eliminar columna 'deck' (porque faltan demasiados datos)
df_limpio = df.dropna(axis=1, thresh=500)

# B) Eliminar filas sin 'age' (en una copia aparte para probar)
df_sin_filas = df.dropna(subset=['age'], axis=0)

# --- 3. IMPUTACIÓN (Rellenar) ---
df_rellenado = df.copy()

# C) Rellenar Edad con Promedio (FORMA MODERNA)
promedio_edad = df['age'].mean()
# Así se hace ahora para evitar el warning:
df_rellenado['age'] = df_rellenado['age'].fillna(promedio_edad)

# D) Rellenar Ciudad con Moda (FORMA MODERNA)
ciudad_comun = df['embark_town'].value_counts().idxmax()
# Así se hace ahora:
df_rellenado['embark_town'] = df_rellenado['embark_town'].fillna(ciudad_comun)

# --- 4. VERIFICACIÓN FINAL ---
print("\n--- Resultado Final (Ceros = Limpio) ---")
# Solo mostramos las columnas que tocamos para no marearte
print(df_rellenado[['age', 'embark_town']].isnull().sum())

--- 1. Datos Faltantes Iniciales ---
survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

--- Resultado Final (Ceros = Limpio) ---
age            0
embark_town    0
dtype: int64


In [4]:
import pandas as pd

# --- 1. CREACIÓN DE DATOS SUCIOS ---
data = {
    'ID': [101, 102, 103, 101, 104, 102],
    'Nombre': ['Ana', 'Beto', 'Carla', 'Ana', 'David', 'Beto'],
    'Email': ['ana@test.com', 'beto@test.com', 'carla@test.com', 'ana@test.com', 'david@test.com', 'beto@nuevo.com']
}

df = pd.DataFrame(data)

print("--- 1. DataFrame Original (Con Duplicados) ---")
print(df)
print("-" * 30)


# --- 2. DETECCIÓN (El Diagnóstico) ---
# duplicated() devuelve True/False.
# duplicated().sum() nos dice cuántos hay.
num_duplicados = df.duplicated().sum()
print(f"Filas duplicadas EXACTAS: {num_duplicados}")

# Truco Pro: Ver cuáles son las duplicadas
print("\n--- Mostrando las filas que son copias ---")
print(df[df.duplicated()]) 
# Nota: Verás que sale el ID 101 (Ana) porque es idéntica a la primera.
# Pero NO sale el ID 102 (Beto) porque su email cambió, así que para Pandas no son idénticos.


# --- 3. LIMPIEZA NIVEL 1: Duplicados Exactos ---
# drop_duplicates() borra solo si TODAS las columnas coinciden
df_limpio_exacto = df.drop_duplicates()

print(f"\n--- 2. Limpieza Exacta (Se fue la copia de Ana) ---")
print(df_limpio_exacto)


# --- 4. LIMPIEZA NIVEL 2: Duplicados por Columna (Subset) ---
# Problema: Beto (102) aparece dos veces con emails distintos. 
# Regla de negocio: "Un ID debe ser único".
# Usamos 'subset' para decirle: "Si el ID se repite, bórralo".

# keep='first' (por defecto): Se queda con el primero que encontró.
# keep='last': Se queda con el último (útil si quieres el dato más reciente).

df_sin_repes_id = df.drop_duplicates(subset=['ID'], keep='first')

print(f"\n--- 3. Limpieza por ID (Se fue el segundo Beto) ---")
print(df_sin_repes_id)

--- 1. DataFrame Original (Con Duplicados) ---
    ID Nombre           Email
0  101    Ana    ana@test.com
1  102   Beto   beto@test.com
2  103  Carla  carla@test.com
3  101    Ana    ana@test.com
4  104  David  david@test.com
5  102   Beto  beto@nuevo.com
------------------------------
Filas duplicadas EXACTAS: 1

--- Mostrando las filas que son copias ---
    ID Nombre         Email
3  101    Ana  ana@test.com

--- 2. Limpieza Exacta (Se fue la copia de Ana) ---
    ID Nombre           Email
0  101    Ana    ana@test.com
1  102   Beto   beto@test.com
2  103  Carla  carla@test.com
4  104  David  david@test.com
5  102   Beto  beto@nuevo.com

--- 3. Limpieza por ID (Se fue el segundo Beto) ---
    ID Nombre           Email
0  101    Ana    ana@test.com
1  102   Beto   beto@test.com
2  103  Carla  carla@test.com
4  104  David  david@test.com


In [ ]:
#ejercicios 0110
import pandas as pd
import seaborn as sns
import numpy as np

# --- 1. CARGAR DATOS (Auto MPG) ---
# En el PDF cargan un CSV, nosotros usaremos el de la librería directa
print("--- 1. Carga de Datos ---")
df = sns.load_dataset('mpg')

# Limpiamos un poco los nulos antes de empezar (como aprendimos en la 3.3)
df.dropna(inplace=True)
print(df.head())


# --- 2. VARIABLES DERIVADAS (Crear información nueva) ---
print("\n--- 2. Variable Derivada: Peso en kg ---")
# La columna 'weight' está en libras. Queremos kg.
df['weight_kg'] = df['weight'] * 0.453592

# Redondeamos a 2 decimales [cite: 704]
df['kpl'] = df['kpl'].round(2)

print(df[['mpg', 'kpl']].head())



--- 1. Carga de Datos ---
    mpg  cylinders  displacement  horsepower  weight  acceleration  \
0  18.0          8         307.0       130.0    3504          12.0   
1  15.0          8         350.0       165.0    3693          11.5   
2  18.0          8         318.0       150.0    3436          11.0   
3  16.0          8         304.0       150.0    3433          12.0   
4  17.0          8         302.0       140.0    3449          10.5   

   model_year origin                       name  
0          70    usa  chevrolet chevelle malibu  
1          70    usa          buick skylark 320  
2          70    usa         plymouth satellite  
3          70    usa              amc rebel sst  
4          70    usa                ford torino  

--- 2. Variable Derivada: KPL ---
    mpg   kpl
0  18.0  7.65
1  15.0  6.38
2  18.0  7.65
3  16.0  6.80
4  17.0  7.22


In [ ]:

# --- 3. TRANSFORMACIÓN DE TIPOS (Categories) ---
print("\n--- 3. Tipos de Datos ---")
print("Antes:", df['origin'].dtype)
df['origin'] = df['origin'].astype('category')
print("Ahora:", df['origin'].dtype)



--- 3. Tipos de Datos ---
Antes: object
Ahora: category


In [4]:


# --- 4. BINNING (Agrupar Caballos de Fuerza) ---
# Vamos a clasificar la potencia ('horsepower') en 3 niveles: Bajo, Medio, Alto.
# Usamos histograma para ver los cortes [cite: 913]
count, bin_dividers = np.histogram(df['horsepower'], bins=3)
bin_names = ['Baja Potencia', 'Media Potencia', 'Alta Potencia']
print("\n--- 4. Binning de Horsepower ---")

#bajo 
df['horsepower_binned'] = pd.cut(df['horsepower'], bins=bin_dividers, labels=bin_names, include_lowest=True)
#medio


#alto
print(df[['horsepower', 'horsepower_binned']].head(10))




--- 4. Binning de Horsepower ---
   horsepower horsepower_binned
0       130.0    Media Potencia
1       165.0    Media Potencia
2       150.0    Media Potencia
3       150.0    Media Potencia
4       140.0    Media Potencia
5       198.0     Alta Potencia
6       220.0     Alta Potencia
7       215.0     Alta Potencia
8       225.0     Alta Potencia
9       190.0     Alta Potencia


In [5]:
# Creamos la columna de rangos con pd.cut [cite: 998]
df['hp_bin'] = pd.cut(x=df['horsepower'],
                      bins=bin_dividers,
                      labels=bin_names,
                      include_lowest=True)

print("\n--- 4. Binning (Rangos) ---")
print(df[['horsepower', 'hp_bin']].sample(5))





--- 4. Binning (Rangos) ---
     horsepower          hp_bin
124       180.0   Alta Potencia
117        49.0   Baja Potencia
233        78.0   Baja Potencia
166       129.0  Media Potencia
375        74.0   Baja Potencia


In [ ]:
# --- 5. DUMMY VARIABLES (One-Hot Encoding) ---
# Las IAs no entienden "Baja Potencia" o "Alta Potencia".
# Convertimos esa columna nueva en números (0 y 1)[cite: 1127].
print("\n--- 5. Dummy Variables ---")
dummies = pd.get_dummies(df['hp_bin'])
print(dummies.head())

# (Opcional) Pegarlo al dataframe original
df = pd.concat([df, dummies], axis=1)


In [ ]:


# --- 6. NORMALIZACIÓN (Escalado) ---
# Vamos a escalar el 'peso' (weight) para que vaya de 0 a 1.
# Fórmula: (Valor - Min) / (Max - Min) [cite: 1225-1227]
print("\n--- 6. Normalización (Peso) ---")
print(f"Peso original (Max): {df['weight'].max()}")

df['peso_normalizado'] = (df['weight'] - df['weight'].min()) / (df['weight'].max() - df['weight'].min())

print(f"Peso normalizado (Max): {df['peso_normalizado'].max()}")
print(df[['weight', 'peso_normalizado']].head())

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# ==============================================================================
# 1. CARGA DE DATOS (Simulando el CSV)
# ==============================================================================
print("--- 1. CARGA DE DATOS ---")
# El PDF usa pd.read_csv('./auto-mpg.csv'). Nosotros usaremos Seaborn que ya lo trae.
df = sns.load_dataset('mpg')

# Para que coincida EXACTAMENTE con el PDF, renombramos columnas (aunque ya vienen bien)
df.columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
              'acceleration', 'model_year', 'origin', 'name']

print(df.head(3))
print("-" * 30)

# ==============================================================================
# 2. LIMPIEZA PREVIA (PDF Lines 9-6 a 9-8)
# ==============================================================================
# El PDF menciona que 'horsepower' tiene signos '?' que hay que borrar.
# En el dataset de Seaborn, ya vienen como NaN, así que solo eliminamos los nulos.
print("\n--- 2. LIMPIEZA (Horsepower) ---")
print(f"Nulos antes: {df['horsepower'].isnull().sum()}")

df.dropna(subset=['horsepower'], axis=0, inplace=True)

# Aseguramos que sea float (decimal)
df['horsepower'] = df['horsepower'].astype('float')

print(f"Nulos después: {df['horsepower'].isnull().sum()}")
print("-" * 30)


--- 1. CARGA DE DATOS ---
    mpg  cylinders  displacement  horsepower  weight  acceleration  \
0  18.0          8         307.0       130.0    3504          12.0   
1  15.0          8         350.0       165.0    3693          11.5   
2  18.0          8         318.0       150.0    3436          11.0   

   model_year origin                       name  
0          70    usa  chevrolet chevelle malibu  
1          70    usa          buick skylark 320  
2          70    usa         plymouth satellite  
------------------------------

--- 2. LIMPIEZA (Horsepower) ---
Nulos antes: 6
Nulos después: 0
------------------------------


In [2]:


# ==============================================================================
# 3. VARIABLE DERIVADA: MPG a KPL (PDF Line 3)
# ==============================================================================
print("\n--- 3. VARIABLE DERIVADA (MPG -> KPL) ---")
# Fórmula del PDF: 1.60934 km/milla / 3.78541 litros/galón
mpg_to_kpl = 1.60934 / 3.78541

# Aplicamos la fórmula (Line 3-3)
df['kpl'] = df['mpg'] * mpg_to_kpl

# Redondeamos a 2 decimales (Line 3-7)
df['kpl'] = df['kpl'].round(2)

print(df[['mpg', 'kpl']].head(3))
print("-" * 30)



--- 3. VARIABLE DERIVADA (MPG -> KPL) ---
    mpg   kpl
0  18.0  7.65
1  15.0  6.38
2  18.0  7.65
------------------------------


In [3]:
# ==============================================================================
# 4. TIPOS DE DATOS (PDF Lines 5 a 8)
# ==============================================================================
print("\n--- 4. CAMBIO DE TIPOS DE DATOS ---")
# El PDF cambia 'origin' y 'model_year' a categoría y luego a string/objeto.
# Esto le dice a la IA: "Oye, el año 70 no es una cantidad, es una etiqueta".

print("Tipo original 'origin':", df['origin'].dtype)
df['origin'] = df['origin'].astype('object') # O 'category'
print("Tipo nuevo 'origin':", df['origin'].dtype)

print("\nTipo original 'model_year':", df['model_year'].dtype)
df['model_year'] = df['model_year'].astype('category')
print("Tipo nuevo 'model_year':", df['model_year'].dtype)
print("-" * 30)


--- 4. CAMBIO DE TIPOS DE DATOS ---
Tipo original 'origin': object
Tipo nuevo 'origin': object

Tipo original 'model_year': int64
Tipo nuevo 'model_year': category
------------------------------


In [4]:
# ==============================================================================
# 5. BINNING: DE NÚMEROS A GRUPOS (PDF Lines 10 a 11)
# ==============================================================================
print("\n--- 5. BINNING (Rangos de Potencia) ---")
# Queremos dividir la potencia en 3 grupos: Bajo, Normal, Alto.

# Paso A: Calcular los cortes (histograma)
count, bin_dividers = np.histogram(df['horsepower'], bins=3)
print("Límites de los grupos:", bin_dividers)

# Paso B: Crear las etiquetas
bin_names = ['Low output', 'Normal output', 'High output']

# Paso C: Cortar los datos (pd.cut)
df['hp_bin'] = pd.cut(x=df['horsepower'],
                      bins=bin_dividers,
                      labels=bin_names,
                      include_lowest=True)

print(df[['horsepower', 'hp_bin']].sample(5))
print("-" * 30)


--- 5. BINNING (Rangos de Potencia) ---
Límites de los grupos: [ 46.         107.33333333 168.66666667 230.        ]
     horsepower         hp_bin
17         85.0     Low output
140       150.0  Normal output
293        71.0     Low output
49         86.0     Low output
36         88.0     Low output
------------------------------


In [5]:
# ==============================================================================
# 6. DUMMY VARIABLES (One-Hot Encoding) (PDF Line 13)
# ==============================================================================
print("\n--- 6. DUMMY VARIABLES (Para la IA) ---")
# Convertimos la columna de texto 'hp_bin' en columnas numéricas (0 y 1)
horsepower_dummies = pd.get_dummies(df['hp_bin'])

print(horsepower_dummies.head(5))
print("-" * 30)


--- 6. DUMMY VARIABLES (Para la IA) ---
   Low output  Normal output  High output
0       False           True        False
1       False           True        False
2       False           True        False
3       False           True        False
4       False           True        False
------------------------------


In [6]:
# ==============================================================================
# 7. NORMALIZACIÓN (Escalado Min-Max) (PDF Lines 14 a 15)
# ==============================================================================
print("\n--- 7. NORMALIZACIÓN (0 a 1) ---")
# Queremos que 'horsepower' vaya de 0 a 1 para no confundir a la red neuronal.

# Estadística antes
print("Máximo original:", df['horsepower'].max())

# Fórmula: (X - Min) / (Max - Min)
min_val = df['horsepower'].min()
max_val = df['horsepower'].max()

df['horsepower_normalized'] = (df['horsepower'] - min_val) / (max_val - min_val)

# Resultado
print("Máximo normalizado:", df['horsepower_normalized'].max())
print(df[['horsepower', 'horsepower_normalized']].head(5))

print("\n\n¡PROCESO TERMINADO! Tu DataFrame ya es apto para Inteligencia Artificial.")


--- 7. NORMALIZACIÓN (0 a 1) ---
Máximo original: 230.0
Máximo normalizado: 1.0
   horsepower  horsepower_normalized
0       130.0               0.456522
1       165.0               0.646739
2       150.0               0.565217
3       150.0               0.565217
4       140.0               0.510870


¡PROCESO TERMINADO! Tu DataFrame ya es apto para Inteligencia Artificial.
